In [1]:
import pandas as pd

df = pd.read_csv('2019-12.csv', encoding='cp949', low_memory=False)

In [122]:
import matplotlib.pyplot as plt
import seaborn as sns

use_category = [1001, 1003, 1004, 1007, 1008, 1010, 1099, 1102, 1104, 1105, 1199,
                1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1299, 
                3001, 3002, 3101, 3102, 3201, 3199, 3202, 3203,
                3299, 6101, 6102, 6103, 6109, 6110, 5502, 5605, 
                6001, 6002, 6003, 6004, 6005, 6006, 6007, 6008, 6009, 6010, 6011, 6013,
                6016, 6021, 6099, 6019, 6020, 6018, 6107, 6113, 6201, 6202,
                6203, 6204, 6205, 6206, 6207, 6208, 6111, 6112, 6299, 
                7004, 7006, 7007, 7008, 7010, 7099, 7101, 7103,
                7106, 7108, 7199, 8108, 8115, 8109, 8110, 8101, 8102,
                8106, 8112, 8113, 8114, 8111, 6114, 6115, 6116, 6105, 6199, 8406]


age_mask = df['연령'] != '0.기타'
gender_mask = (df['성별'] != '0.기타') & (df['성별'] != '3.기업')
store_mask = df['가맹점업종코드'].isin(use_category)
df = df[age_mask & gender_mask & store_mask]

group_data = (df.groupby(['성별', '연령', '가맹점업종코드'])['결제금액', '회원수', '결제건수']
              .sum()
              .reset_index())

gender_data = group_data['성별'].apply(lambda x: int(x[0]))
age_data = group_data['연령'].apply(lambda x: int(x[0]))
store_data = group_data['가맹점업종코드']
cash_avg_data = group_data['결제금액'] // group_data['결제건수']
count_data = group_data['회원수']

agg_data = pd.DataFrame({
    'gender': gender_data,
    'age': age_data,
    'store': store_data,
    'cash': cash_avg_data,
    'count': count_data})

agg_data['perc_count'] = agg_data.groupby(['gender', 'age']).apply(lambda g: g / g.sum())['count'] * 100
print(agg_data)


# test_df = df[store_mask]
# test2_df = test_df.groupby('가맹점업종코드')['결제건수'].sum().reset_index()
# test_dict = test2_df.set_index('가맹점업종코드').to_dict()
# agg_data['전체 결제건수'] = agg_data['store'].apply(lambda x: test_dict['결제건수'][x])
# agg_data['테스트'] = agg_data['count'] / agg_data['전체 결제건수'] * 100
# print(agg_data)

     age    cash  count  gender  store  perc_count
0      1   44393    962       1   1004    0.265485
1      1  149042     14       1   1010    0.003864
2      1   49700      3       1   1099    0.000828
3      1   15088     64       1   1201    0.017662
4      1   12000      3       1   1204    0.000828
..   ...     ...    ...     ...    ...         ...
731    7  139729    409       2   7106   11.875726
732    7   42250     20       2   8101    0.580720
733    7    7000      3       2   8113    0.087108
734    7   30653    242       2   8114    7.026713
735    7   13535      6       2   8406    0.174216

[736 rows x 6 columns]


In [123]:
pd.set_option('display.max_row', 500)
category_data = pd.read_csv('category.csv').set_index('code').to_dict()
gender_data = {1: '남성', 2 :'여성'}
def change_age(n):
    return str(n) + '0대'

sort_by_count_values = (agg_data.groupby(['gender', 'age'])['store', 'cash', 'count', 'perc_count']
                       .apply(lambda x: x.sort_values(by='perc_count', ascending=False)[:5])
                       .reset_index()
                       .drop('level_2', axis=1))
sort_by_count_values['gender'] = sort_by_count_values['gender'].apply(lambda x: gender_data[x])
sort_by_count_values['age'] = sort_by_count_values['age'].apply(lambda x: change_age(x))
sort_by_count_values['업종'] = sort_by_count_values['store'].apply(lambda x: category_data['name'][x])
# sort_by_count_values['age'] = sort_by_count_values['count'].apply(lambda x: change_age(x))
# sort_by_count_values['설명'] = sort_by_count_values['store'].apply(lambda x: category_data['description'][x])

sort_by_cash_values = (agg_data.groupby(['gender', 'age'])['store', 'cash', 'count']
                       .apply(lambda x: x.sort_values(by='cash', ascending=False)[:5])
                      .reset_index()
                      .drop('level_2', axis=1))
sort_by_cash_values['업종'] = sort_by_cash_values['store'].apply(lambda x: category_data['name'][x])
# sort_by_cash_values['설명'] = sort_by_cash_values['store'].apply(lambda x: category_data['description'][x])

print(sort_by_count_values.set_index(['gender', 'age', '업종'])['perc_count'])
# print(sort_by_cash_values)

gender  age  업종           
남성      10대  비디오방/게임방         98.172515
             문방구점              0.315988
             컴퓨터기기             0.309640
             서점                0.299153
             기성복점              0.265485
        20대  비디오방/게임방         91.550047
             기성복점              1.732918
             화장품점              1.649187
             문방구점              1.447380
             완구점               1.011302
        30대  비디오방/게임방         40.673793
             기성복점              8.966716
             문방구점              7.195353
             서점                6.946687
             가전제품점             6.809153
        40대  서점               16.523406
             학습지              13.096649
             문방구점             10.907890
             문리계학원             8.603517
             기성복점              8.520936
        50대  서점               13.936896
             기성복점             10.395033
             차량 정비/부품/인테리어    10.229831
             문방구점              8.962675
             

In [124]:
male_df = df[df['성별'] == '1.남성']
female_df = df[df['성별'] == '2.여성']
male_group_data = male_df.groupby(['연령', '가맹점업종코드'])['결제금액', '회원수', '결제건수'].sum().reset_index()
female_group_data = female_df.groupby(['연령', '가맹점업종코드'])['결제금액', '회원수', '결제건수'].sum().reset_index()

print(male_group_data)
male_age_data = male_group_data['연령'].apply(lambda x: int(x[0]))
male_store_data = male_group_data['가맹점업종코드']
male_cash_data = male_group_data['결제금액'] // male_group_data['결제건수']
male_count_data = male_group_data['회원수']

male_final_data = pd.DataFrame({
    'age': male_age_data,
    'store': male_store_data,
    'cash': male_cash_data,
    'count': male_count_data})

print('남성 고객들이 특정 업종에서 평균적으로 사용하는 금액')
male_cash = male_final_data.dropna().pivot(index='store', columns='age', values='cash').fillna(0)
print(male_cash)


print('10대 남성들이 특정 업종에서 평균적으로 사용하는 금액')
print(male_cash[1].sort_values())
print('-' * 100)

print('남성 고객들이 특정 업종에서 평균적으로 결제하는 횟수')
print(male_final_data.dropna().pivot(index='store', columns='age', values='count').fillna(0))

          연령  가맹점업종코드        결제금액      회원수     결제건수
0      1.10대     1004    45636005      962     1028
1      1.10대     1010     2086600       14       14
2      1.10대     1099      149100        3        3
3      1.10대     1201      995850       64       66
4      1.10대     1204       36000        3        3
5      1.10대     1207       80000        3        3
6      1.10대     3001       83000       31       32
7      1.10대     3101      132900       10       13
8      1.10대     3199      121300       12       13
9      1.10대     3203       28300        3        3
10     1.10대     6001     4803750      108      119
11     1.10대     6002      233600       13       13
12     1.10대     6007     7672100      532      550
13     1.10대     6010      164600       24       26
14     1.10대     6011     1919250      256      261
15     1.10대     6013       93500       12       12
16     1.10대     6018  1274621934   355734   399052
17     1.10대     6019       62500        6        6
18     1.10대